<a href="https://colab.research.google.com/github/FED32/Diplom_netology/blob/master/04_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 36.3 MB/s 
     |████████████████████████████████| 6.8 MB 33.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import warnings
import numpy as np
import transformers
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm_notebook
# from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Add, dot, LSTM
warnings.filterwarnings("ignore")

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
torch.cuda.is_available()

False

https://github.com/google-research/bert

https://huggingface.co/transformers/model_doc/bert.html#bertmodel

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# считываем датасеты с нормированными описаниями тендеров

train_data_desc_stemmed = pd.read_csv('gdrive/MyDrive/data/train_data_desc_stemmed.csv')
test_data_desc_stemmed = pd.read_csv('gdrive/MyDrive/data/test_data_desc_stemmed.csv')

# соединяем датасеты

data = pd.concat([train_data_desc_stemmed, test_data_desc_stemmed], ignore_index=True)
print(data.shape)

(862999, 2)


In [ ]:
data = data[862000:]
data

,pn_lot_id,text_description_tender_stemmed
862000,9848325,запрос предложение электронный форма участник ...
862001,1716910,право заключение договор подряд выполнение стр...
862002,4604740,поставка провод лот кабель электронный электри...
862003,3011923,оказание услуга рекламный информационный сопро...
862004,777635,определение подрядчик выполнение смр пнр обору...
...,...,...
862994,5373821,право заключение договор выполнение строительн...
862995,8396902,право заключение договор выполнение строительн...
862996,7569089,запрос ценовый предложение электронный форма п...
862997,2403905,право заключение договор выполнение проектный ...


In [ ]:
# инициализация модели
tokenizer = transformers.BertTokenizer(vocab_file='gdrive/MyDrive/download_models/uncased_L-12_H-768_A-12/vocab.txt')
config = transformers.BertConfig.from_json_file('gdrive/MyDrive/download_models/uncased_L-12_H-768_A-12/bert_config.json')
model = transformers.BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# tokenized = data['text_description_tender_stemmed'].apply(lambda x: tokenizer.encode(x, max_length=100, add_special_tokens = True))

tokenized = []
for str in tqdm_notebook(data['text_description_tender_stemmed']):
  tokenized.append(tokenizer.encode(str, max_length=100, add_special_tokens = True))

# print("Tokenized head:", tokenized.head())
print("Tokenized head:", pd.DataFrame(tokenized).head())

n = max(map(len, tokenized))  # применим padding (уравняем длины исходных описаний)
for i in range(len(tokenized)):   
    tokenized[i] = tokenized[i] + [0]*(n-len(tokenized[i]))
tokenized = np.stack(tokenized)
attention_mask = np.where(tokenized != 0, 1, 0) # создадим маску (укажем нулевые и ненулевые значения)
print("Attention mask:", attention_mask.shape)

# сделаем цикл по батчам:
batch_size = 999
embeddings = []
for i in tqdm_notebook(range(tokenized.shape[0] // batch_size)):       
    batch = torch.LongTensor(tokenized[batch_size*i:batch_size*(i+1)])  # преобразуем данные в формат тензоров
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])  # преобразуем маску
    with torch.no_grad(): # градиенты не нужны, модель BERT обучать не будем
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        # получаем эмбеддинги для батча, передав модели данные и маску
    embeddings.append(batch_embeddings[0][:,0,:].numpy())  
        # извлекаем нужные элементы из тензора и добавляем в список эмбеддингов  

  0%|          | 0/999 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Tokenized head:     0     1      2      3      4   ...       95       96       97       98     99
0  101  1187  10260  29746  16856  ...  16856.0   1194.0  14150.0  29747.0  102.0
1  101  1194  16856  10260  25529  ...  29436.0   1193.0  29740.0  14150.0  102.0
2  101  1194  14150  29747  22919  ...      NaN      NaN      NaN      NaN    NaN
3  101  1193  28598  29744  28995  ...  18947.0  18947.0  14150.0   1189.0  102.0
4  101  1193  29746  16856  15290  ...  29436.0   1194.0  10260.0  14150.0  102.0

[5 rows x 100 columns]
Attention mask: (999, 100)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
vectors = np.concatenate(embeddings)
print(vectors.shape)

(999, 768)


In [ ]:
X_reduced = TruncatedSVD(n_components=100, random_state=0).fit_transform(vectors) # оставляем n главных компонент
bert_vectors_redused  = pd.DataFrame(data['pn_lot_id']).join(pd.DataFrame(X_reduced).set_index(data['pn_lot_id'].index))
# bert_vectors_redused  = pd.DataFrame(data['pn_lot_id']).merge((pd.DataFrame(X_reduced).set_index(data['pn_lot_id'])), on='pn_lot_id')

In [ ]:
bert_vectors_redused

,pn_lot_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
862000,9848325,13.348967,0.100945,-0.024083,-0.333258,-0.192029,-0.495873,1.044093,-0.195336,-0.146591,-0.316052,0.330057,0.660057,-0.181432,0.144290,0.475612,-0.237896,0.347093,-0.246241,-0.360688,0.168650,0.207438,-0.059604,-0.099415,-0.471580,0.298001,-0.006727,-0.123064,0.074318,-0.319241,-0.135975,-0.033202,0.011663,-0.023328,0.112558,0.166359,0.142299,-0.083196,-0.148535,0.197880,...,0.054213,-0.106128,0.013293,-0.099341,-0.137101,0.007927,0.106949,0.013041,-0.092258,0.093942,-0.077431,0.012250,0.013775,0.098016,-0.039835,-0.021008,-0.083292,0.028535,-0.178921,0.001926,-0.042052,0.051797,-0.118854,0.076941,0.011575,0.019674,-0.057711,0.089719,0.030219,-0.091237,-0.055382,-0.041025,0.020920,0.074026,-0.109227,-0.074054,-0.010794,-0.116635,-0.031970,-0.040315
862001,1716910,13.873367,0.346565,-0.666118,0.517051,0.983317,0.165231,0.288056,0.002526,-0.801079,0.105886,-0.404894,0.207926,-0.063257,0.100446,-0.129142,-0.165511,0.004712,-0.151653,0.104068,0.092421,0.462049,0.097492,0.027795,0.131414,0.098013,0.024964,0.084451,-0.116006,-0.221862,-0.177193,0.186991,0.432108,0.112754,0.403317,-0.148218,-0.033746,-0.195741,-0.025321,0.023781,...,0.119163,-0.093185,0.047555,0.052234,-0.172100,-0.121972,0.052413,-0.247769,0.129780,-0.129809,-0.071742,-0.014110,0.000262,0.169085,-0.068264,0.014091,-0.087739,0.115140,0.027254,-0.023247,-0.103323,-0.131556,-0.039466,0.053437,0.173978,-0.173528,0.075397,0.057513,-0.032642,-0.031489,-0.153073,-0.060933,-0.058265,0.069025,0.075204,-0.037931,-0.163929,-0.063118,-0.047295,-0.016413
862002,4604740,13.824291,-1.732929,-0.669764,0.374998,0.373451,0.326555,-0.756872,0.183618,-0.115765,-0.210653,0.126686,-0.171635,-0.095591,-0.033019,-0.042453,-0.346724,0.105725,-0.021331,-0.090725,0.404196,-0.237893,-0.167976,0.002070,0.017665,0.128009,0.248000,0.091051,0.252285,-0.103401,0.059740,-0.179901,-0.045245,-0.234355,0.190698,-0.045885,0.085144,-0.155729,-0.029525,0.121110,...,0.061715,0.004377,0.065853,-0.027573,-0.092716,0.128206,-0.010632,0.138975,-0.023996,-0.104309,-0.025428,-0.059795,0.026663,-0.268749,-0.031498,0.140181,0.048267,0.075988,0.032708,0.093281,0.028960,-0.049328,0.107731,-0.075267,0.022651,-0.001918,0.003832,0.046145,-0.008262,-0.019415,-0.014812,-0.067914,0.105839,0.013072,0.006108,0.046791,0.043190,-0.040173,0.039877,0.062886
862003,3011923,13.906985,-0.287845,1.134231,0.966541,-0.358791,-0.582481,-0.495519,0.153251,-0.188183,-0.339812,0.284663,0.410009,0.459500,0.512975,-0.025959,-0.066457,0.133983,-0.163721,-0.277606,-0.124578,-0.123868,0.007310,-0.060214,0.421638,0.113232,-0.241653,0.081753,0.281306,0.516944,0.246726,0.366442,0.281838,0.033646,-0.095890,-0.013130,0.064025,-0.075319,-0.137236,0.148134,...,0.260955,-0.027990,0.009389,-0.097949,-0.177678,0.026429,-0.059380,0.072385,-0.080979,-0.034179,-0.068107,0.175540,-0.008435,-0.027811,-0.017606,-0.092193,0.174944,-0.013539,-0.032002,-0.244428,0.081642,-0.126724,-0.084690,0.088525,-0.024336,0.135770,-0.028660,0.010708,0.062665,-0.055547,-0.152872,-0.182581,-0.072942,0.003214,0.149376,-0.078900,-0.235076,0.038533,0.130322,-0.132722
862004,777635,13.401104,1.443199,0.220178,-0.174993,0.202275,-0.813035,0.155526,0.130011,-0.371908,0.159924,0.314138,0.210696,-0.579936,-0.136774,-0.204171,-0.015208,-0.261424,0.029578,0.154734,-0.053865,-0.089394,0.026574,-0.243520,-0.265333,0.261350,0.138477,0.040420,-0.158889,-0.063362,0.118513,0.016131,0.068537,0.116910,0.023075,-0.049340,-0.028558,0.117373,-0.025819,0.123186,...,0.084408,0.163198,-0.026773,0.018798,0.251394,0.047684,-0.094536,-0.005114,0.149064,-0.059284,0.029975,0.049373,-0.096749,-0.006666,0.107318,0.070886,-0.011520,-0.049289,-0.051579,0.046102,0.117057,-0.002773,-0.030998,-0.020707,0.015369,-0.017519,-0.004985,-0.046226,-0.034744,0.047954

In [ ]:
bert_vectors_redused.to_csv('gdrive/MyDrive/data/bert_vectors_redused(862000-).csv', index=False, encoding='utf-8')

In [ ]:
# data

In [ ]:
# pd.DataFrame(X_reduced).set_index(data['pn_lot_id'].index)

In [ ]:
# data['pn_lot_id'].index

In [ ]:
# pd.DataFrame(X_reduced).set_index(data['pn_lot_id'])